In [ ]:
!pip uninstall -y jax jaxlib

Found existing installation: jax 0.4.26
Uninstalling jax-0.4.26:
  Successfully uninstalled jax-0.4.26
Found existing installation: jaxlib 0.4.26+cuda12.cudnn89
Uninstalling jaxlib-0.4.26+cuda12.cudnn89:
  Successfully uninstalled jaxlib-0.4.26+cuda12.cudnn89


In [ ]:
!pip install -Uqq datasets optuna kaleido

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 10.5 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata, runtime
import subprocess

In [ ]:
hf_token = userdata.get('hf_token')
input_str = f'{hf_token}\nn\n'
result = subprocess.run(['huggingface-cli', 'login'], input=input_str, text=True, capture_output=True)
print(result.stdout)


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Add token as git credential? (Y/n) Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful



In [ ]:
from functools import reduce
import os
import random
import colorsys
import math
import numpy as np
import pandas as pd
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, default_collate
from torchvision.transforms import Resize, Normalize, ToTensor, Compose, transforms, CenterCrop, RandomCrop, RandomChoice
from sklearn.metrics import f1_score
import optuna
import pickle
import json

import torchvision.io as io

from datasets import load_dataset, concatenate_datasets

# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

focus = 'synthetic_properties'
results_dir = f'/content/drive/MyDrive/Colab_Notebooks/dye_test_opt/ternary/results/{focus}'

n_trials = 150

# Load dataset
ds = load_dataset('mpg-ranch/dye_test', split='train')

with open(os.path.join(results_dir.replace(focus,'learning'),'study.pkl'), 'rb') as f:
    learning_study = pickle.load(f)
    best_learning_params = learning_study.best_params

# Dataset preparation
total_samples = len(ds)
examples_per_set = 88 #multiplied by 3x months later
n_workers = os.cpu_count()
print(f'Number of available CPU cores: {n_workers}')
n_epochs = 10
n_seeds = 3
context_sz = 154 # 1x1 meter context
canvas_sz = context_sz + 14*6 # 1x1 meter context
bs = best_learning_params.get('bs')
lr = best_learning_params.get('lr')
blue_wt = best_learning_params.get('blue_wt')
red_wt = best_learning_params.get('red_wt')

print(f"Best parameters: lr={lr}, bs={bs}")

imagenet_stats = {'mean': [0.485, 0.456, 0.406],
                  'std': [0.229, 0.224, 0.225]}

# Preprocessing transforms
preprocs = Compose([
    CenterCrop((context_sz, context_sz))
])

def preproc_transforms(examples):
    examples["img"] = [preprocs(image.convert("RGB")) for image in examples["image"]]
    return examples

print("Applying preprocessing transforms...")
ds = ds.map(preproc_transforms, remove_columns=["image","color","size","concentration"], batched=True, batch_size=len(ds))
n_classes = len(np.unique(ds['label']))

def filter_month(batch, month):
    """Returns a list of booleans indicating whether each example in the batch should be kept."""
    # Assuming 'month' is a key in your dataset's dictionary and it's stored in a list for each batch
    return [example_month == month for example_month in batch['month']]

# Define model loading function
def load_model(arch, n_classes):
    print("Loading model...")
    model = torch.hub.load('facebookresearch/dinov2', arch)
    num_ftrs = model.norm.normalized_shape[0]
    model.head = nn.Linear(num_ftrs, n_classes)
    model.to(device)
    return model

def modified_f1_score(labels, predictions):
    scores = []

    # For 1 vs. 0, exclude class 2 and consider only 1 and 0 for binary comparison
    mask_1_vs_0 = (labels != 2)
    labels_1_vs_0 = labels[mask_1_vs_0] == 1
    predictions_1_vs_0 = predictions[mask_1_vs_0] == 1
    if np.any(labels_1_vs_0) or np.any(predictions_1_vs_0):
        f1_1_vs_0 = f1_score(labels_1_vs_0, predictions_1_vs_0, pos_label=True, average='binary')
        scores.append(f1_1_vs_0)

    # For 2 vs. 0, exclude class 1 and consider only 2 and 0 for binary comparison
    mask_2_vs_0 = (labels != 1)
    labels_2_vs_0 = labels[mask_2_vs_0] == 2
    predictions_2_vs_0 = predictions[mask_2_vs_0] == 2
    if np.any(labels_2_vs_0) or np.any(predictions_2_vs_0):
        f1_2_vs_0 = f1_score(labels_2_vs_0, predictions_2_vs_0, pos_label=True, average='binary')
        scores.append(f1_2_vs_0)

    # Calculate the mean of the F1 scores if any valid scores were calculated
    mean_f1 = np.mean(scores) if scores else 0.0

    return f1_1_vs_0, f1_2_vs_0, mean_f1

class SuperimposeSquare(object):
    def __init__(self, red_hue=0.83, blue_hue=0.45,
                 red_value=0.4, blue_value=0.4,
                 red_saturation=0.4, blue_saturation=0.4,
                 max_opacity=0.3, min_opacity=0.1):
        self.red_hue = red_hue
        self.blue_hue = blue_hue
        self.red_value = red_value
        self.blue_value = blue_value
        self.red_saturation = red_saturation
        self.blue_saturation = blue_saturation
        self.max_opacity = max(0, min(1, max_opacity))
        self.min_opacity = max(0, min(1, min_opacity))

    def __call__(self, tensor):
      image = tensor.unsqueeze(0)
      h, w = image.size()[-2:]

      # Randomly choose between small and large box sizes
      small_box = random.choice([True, False])
      if small_box:
          mask_size = 15
      else:
          mask_size = 77

      color_choice = random.choice(['blue', 'red'])
      if color_choice == 'red':
          hue = self.red_hue
          value = self.red_value
          saturation = self.red_saturation
          label = 2
      else:
          hue = self.blue_hue
          value = self.blue_value
          saturation = self.blue_saturation
          label = 1

      saturation = 1.0  # Full saturation for vivid colors
      color_rgb = colorsys.hsv_to_rgb(hue, saturation, value)
      color_tensor = torch.tensor(color_rgb)

      x = (w - mask_size) // 2  # Centering the square on x-axis
      y = (h - mask_size) // 2  # Centering the square on y-axis

      square = color_tensor.view(3, 1, 1).expand(-1, mask_size, mask_size)
      opacity = random.uniform(self.min_opacity, self.max_opacity)
      square = opacity * square + (1 - opacity) * image[:, :, y:y+mask_size, x:x+mask_size]
      image[:, :, y:y+mask_size, x:x+mask_size] = square

      return image.squeeze(0), label

# Define Optuna objective function
def objective(trial):
    print(f"Starting trial: {trial.number}")

    red_hue = trial.suggest_float("red_hue", 0.75, 1.0, step=0.01)
    blue_hue = trial.suggest_float("blue_hue", 0.4, 0.6, step=0.01)
    red_value = trial.suggest_float("red_value", 0.5, 1.0, step=0.01)
    blue_value = trial.suggest_float("blue_value", 0.5, 1.0, step=0.01)
    red_saturation = trial.suggest_float("red_saturation", 0, 1.0, step=0.01)
    blue_saturation = trial.suggest_float("blue_saturation", 0, 1.0, step=0.01)
    max_opacity = trial.suggest_float("max_opacity", 0.01, 0.5, step=0.01)
    min_opacity = trial.suggest_float("min_opacity", 0.0, max_opacity, step=0.01)

    datasets_by_month = {
        month: ds.filter(function=filter_month, batched=True, batch_size=len(ds), fn_kwargs={'month': month})
        for month in ['may', 'july', 'october']
    }

    # Train transforms
    train_tfms = Compose([
        CenterCrop(size=context_sz),
        #ToTensor applied in collate_fn
        Normalize(mean=imagenet_stats['mean'], std=imagenet_stats['std'])  # Normalize using ImageNet stats
    ])

    val_tfms = Compose([
        CenterCrop(size=context_sz),
        ToTensor(),
        Normalize(mean=imagenet_stats['mean'], std=imagenet_stats['std'])  # Normalize using ImageNet stats
    ])

    def batch_tfms_val(examples):
        examples["img"] = [val_tfms(image) for image in examples["img"]]
        return examples

    f1_scores = {epoch: [] for epoch in range(1, n_epochs + 1)}

    # Seed loop
    for seed in range(n_seeds):
        print(f"Seed: {seed}")
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

        train_ds_list = []
        val_ds_list = []

        # Sample examples_per_set for training and validation from each month's dataset
        for month, dataset in datasets_by_month.items():
            total_samples = len(dataset)
            all_indices = list(range(total_samples))
            sampled_indices = random.sample(all_indices, 2 * examples_per_set)
            train_indices = sampled_indices[:examples_per_set]
            val_indices = sampled_indices[examples_per_set:examples_per_set*2]

            train_ds_month = dataset.select(train_indices)
            val_ds_month = dataset.select(val_indices)

            train_ds_list.append(train_ds_month)
            val_ds_list.append(val_ds_month)

        # Combine the training and validation sets across months
        train_ds_combined = reduce(lambda x, y: concatenate_datasets([x, y]), train_ds_list)
        val_ds_combined = reduce(lambda x, y: concatenate_datasets([x, y]), val_ds_list)

        train_ds_combined = train_ds_combined.filter(lambda x: x['label'] == 0)

        val_ds_combined.set_transform(batch_tfms_val)

        # Model, optimizer, and loss function setup
        model = load_model('dinov2_vitb14', n_classes)
        optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

        criterion = nn.CrossEntropyLoss().to(device)

        def collate_fun(batch):
              imgs = [ToTensor()(item['img']) for item in batch]  # Convert PIL images to tensors
              imgs = torch.stack(imgs)  # Stack all image tensors
              labels = torch.tensor([item['label'] for item in batch])  # Create a tensor of all labels

              new_imgs = []
              new_labels = []

              for img, label in zip(imgs, labels):
                  if label == 0 and 0.5 > random.random():
                      transformed_img, new_label = SuperimposeSquare(red_hue=red_hue, blue_hue=blue_hue,
                                                                     red_value=red_value, blue_value=blue_value,
                                                                     red_saturation=red_saturation, blue_saturation=blue_saturation,
                                                                     max_opacity=max_opacity, min_opacity=min_opacity)(img)
                      transformed_img = train_tfms(transformed_img)
                      new_imgs.append(transformed_img)
                      new_labels.append(new_label)
                  else:
                      img = train_tfms(img)
                      new_imgs.append(img)
                      new_labels.append(label)

              imgs = torch.stack(new_imgs)
              labels = torch.tensor(new_labels)
              # Wrap the result in a dictionary
              return {'img': imgs, 'label': labels}

        train_loader = DataLoader(train_ds_combined, batch_size=bs, shuffle=True, num_workers=n_workers, collate_fn=collate_fun)
        val_loader = DataLoader(val_ds_combined, batch_size=bs, num_workers=n_workers)

        # Training loop
        for epoch in range(1, n_epochs +1):
            print(f"Epoch: {epoch}/{n_epochs}")
            model.train()
            for _, data in enumerate(train_loader):
                inputs, labels = data['img'].to(device), data['label'].to(device)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

            # Validation loop
            model.eval()
            all_labels, all_predictions = [], []
            with torch.no_grad():
                for data in val_loader:
                    inputs, labels = data['img'].to(device), data['label'].to(device)
                    outputs = model(inputs)
                    _, predicted = torch.max(outputs.data, 1)
                    all_labels.extend(labels.cpu().numpy())
                    all_predictions.extend(predicted.cpu().numpy())

            # Compute F1 score
            blue, red, f1 = modified_f1_score(np.array(all_labels), np.array(all_predictions))
            print(f"Blue F1 score: {blue:.4f}\nRed F1 score: {red:.4f}\nMean F1 score: {f1:.4f}")

            f1_scores[epoch].append(f1)

    # Return the mean F1 score over the seeds
    mean_f1_scores = {epoch: np.mean(scores) for epoch, scores in f1_scores.items()}
    highest_f1_epoch, highest_f1_score = max(mean_f1_scores.items(), key=lambda x: x[1])

    print(f"Highest F1-score {highest_f1_score} at epoch {highest_f1_epoch}")

    return highest_f1_score

def study_cb(study, trial):
    patience = 50

    # Directory where the figures and trial data will be saved
    os.makedirs(results_dir, exist_ok=True)

    # Save figures
    if trial.state == optuna.trial.TrialState.COMPLETE:
        opt_hist_path = os.path.join(results_dir, f'history_plot.png')
        fig1 = optuna.visualization.plot_optimization_history(study)
        fig1.write_image(opt_hist_path)

        opt_slice_path = os.path.join(results_dir, f'slice_plot.png')
        fig2 = optuna.visualization.plot_slice(study)
        fig2.write_image(opt_slice_path)

    study_path = os.path.join(results_dir, 'study.pkl')
    with open(study_path, 'wb') as f:
        pickle.dump(study, f)

    if 'best_trial_number' not in study.user_attrs:
        # Initialize with the first trial
        study.set_user_attr('best_trial_number', 0)

    current_best_trial = study.best_trial.number
    best_trial_number = study.user_attrs['best_trial_number']

    # Update the best trial number in study attributes if the current best is newer
    if current_best_trial > best_trial_number:
        study.set_user_attr('best_trial_number', current_best_trial)
        best_trial_number = current_best_trial

    # Check if no improvement has been made for 'tolerance' trials
    if trial.number >= best_trial_number + patience:
        study.stop()


study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=0))
study.optimize(objective, n_trials=n_trials, callbacks=[study_cb])

runtime.unassign()

Using device: cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Number of available CPU cores: 8
Best parameters: lr=8.113530089356352e-06, bs=96
Applying preprocessing transforms...


...

Epoch: 1/10
Blue F1 score: 0.0000
Red F1 score: 0.2456
Mean F1 score: 0.1228
Epoch: 2/10
Blue F1 score: 0.2687
Red F1 score: 0.0444
Mean F1 score: 0.1566
Epoch: 3/10
Blue F1 score: 0.3793
Red F1 score: 0.3077
Mean F1 score: 0.3435
Epoch: 4/10
Blue F1 score: 0.0000
Red F1 score: 0.7222
Mean F1 score: 0.3611
Epoch: 5/10
Blue F1 score: 0.0000
Red F1 score: 0.5714
Mean F1 score: 0.2857
Epoch: 6/10
Blue F1 score: 0.5172
Red F1 score: 0.7606
Mean F1 score: 0.6389
Epoch: 7/10
Blue F1 score: 0.7200
Red F1 score: 0.7857
Mean F1 score: 0.7529
Epoch: 8/10
Blue F1 score: 0.8108
Red F1 score: 0.7312
Mean F1 score: 0.7710
Epoch: 9/10
Blue F1 score: 0.7945
Red F1 score: 0.7952
Mean F1 score: 0.7949
Epoch: 10/10


[I 2024-04-12 12:51:46,841] Trial 83 finished with value: 0.8344363298911487 and parameters: {'red_hue': 1.0, 'blue_hue': 0.56, 'red_value': 0.91, 'blue_value': 0.72, 'red_saturation': 0.49, 'blue_saturation': 0.79, 'max_opacity': 0.27, 'min_opacity': 0.01}. Best is trial 35 with value: 0.8797921468025885.


Blue F1 score: 0.8158
Red F1 score: 0.8354
Mean F1 score: 0.8256
Highest F1-score 0.8344363298911487 at epoch 10
Starting trial: 84
Seed: 0
Loading model...


/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:685: UserWarning:

The distribution is specified by [0.0, 0.29000000000000004] and step=0.01, but the range is not divisible by `step`. It will be replaced by [0.0, 0.29].

Using cache found in /root/.cache/torch/hub/facebookresearch_dinov2_main


Epoch: 1/10
Blue F1 score: 0.3607
Red F1 score: 0.0000
Mean F1 score: 0.1803
Epoch: 2/10
Blue F1 score: 0.0000
Red F1 score: 0.2857
Mean F1 score: 0.1429
Epoch: 3/10
Blue F1 score: 0.0000
Red F1 score: 0.1905
Mean F1 score: 0.0952
Epoch: 4/10
Blue F1 score: 0.6286
Red F1 score: 0.1017
Mean F1 score: 0.3651
Epoch: 5/10
Blue F1 score: 0.6575
Red F1 score: 0.0702
Mean F1 score: 0.3639
Epoch: 6/10
Blue F1 score: 0.6286
Red F1 score: 0.2222
Mean F1 score: 0.4254
Epoch: 7/10
Blue F1 score: 0.6479
Red F1 score: 0.5641
Mean F1 score: 0.6060
Epoch: 8/10
Blue F1 score: 0.6479
Red F1 score: 0.6667
Mean F1 score: 0.6573
Epoch: 9/10
Blue F1 score: 0.7027
Red F1 score: 0.6824
Mean F1 score: 0.6925
Epoch: 10/10
